# Features
## Abstract
- ここでは、点群処理に欠かせないfeatureについて紹介します。

## Introduction
点群を用いたタスクをこなす場合、特徴抽出を用いる手法を使うことでより簡易にタスクが解ける可能性があります。

特徴抽出を用いる手法を利用する場合、手法は以下の様な流れを持ちます。特徴抽出は、点群の点に含まれている座標値や法線などを点群が持っている情報を参照し、それらの情報からタスクを解くために役立つ情報(特徴)を算出します。その後、特徴は目的のタスクの結果を出力するpostprocessに使用されます。

![processing](img/processing.png)

この特徴抽出及び特徴の背景については[Rusu, 2010]のSection 4冒頭で説明されています。以下は、その文章を丸々引用したものとなります。

> In their native representation, points as defined in the concept of 3D mapping systems are simply represented using their Cartesian coordinates $x, y, z$, with respect to a given origin. Assuming that the origin of the coordinate system does not change over time, there could be two points $p_1$ and $p_2$ , acquired at $t_1$ and $t_2$ , having the same coordinates. Comparing these points however is an ill-posed problem, because even though they are equal with respect to some distance measure (e.g. Euclidean metric), they could be sampled on completely different surfaces, and thus represent totally different information when taken together with the other surrounding points in their vicinity. That is because there are no guarantees that the world has not changed between $t_1$ and $t_2$. Some acquisition devices might provide extra information for a sampled point, such as an intensity or surface remission value, or even a color, however that does not solve the problem completely and the comparison remains ambiguous.

> Applications which need to compare points for various reasons require better characteristics and metrics to be able to distinguish between geometric surfaces. The concept of a 3D point as a singular entity with Cartesian coordinates therefore disappears, and a new concept, that of local descriptor takes its place. The literature is abundant of different naming schemes describing the same conceptualization, such as shape descriptors or geometric features but for the remaining of this document they will be referred to as point feature representations.

上記の引用の要約(重要な点)は以下の通りです。また本チュートリアルでは、これら特徴のことを引用元と同じくpoint feature representationと呼ぶこととします。
- 点群中の点を見分けるために識別的な特性を持ったpoint feature representationが必要となる。

本sectionでは、点群処理手法ごとにpoint feature representationの獲得とタスクへの利用の各説明を行います。点群処理手法を2種類、上から順に説明して行きます。

- (1) Methods with handcrafted feature
- (2) Methods with Deep learning


## Methods with handcrafted feature
Handcrafted featureはある点とその他いくつかの点を一組のグループとして扱い、グループ内の点の関係を特徴として使用します。例えば、Point Feature Histogram (PFH) [Rusu et al, 2008]は、以下の処理がなされています。
1. ある点$p_q$を中心として半径$r$以内の点群を近傍点とする。
2. 近傍点同士で2点一組の組み合わせをすべて計算する。下図の左では、二点の組み合わせをエッジによって示している。
3. 各組で、2点間の特徴計算を行う。下図の右より、$(\alpha, \theta, \phi, p_t - p_s)$が点間の特徴となる。
4. 点間の特徴をヒストグラム化する。このヒストグラムがpoint feature representationとなる。
5. 1~4までの手順を任意の点に対して行うことで、任意の点ごとにpoint feature representationを取得できる。

![region](img/pfh.png)  
([Point Cloud Library]より引用と編集)

この様に、ある点と他の点の座標や法線の相対値に基づいてpoint feature representationを作成することで、Handcrafted featureの一つであるPFHを作成します。作成したpoint feature representationは、RegistrationやObject detectionに利用されます。

実際にどの様にして特徴が作成されてタスクの結果として出力されるか、例として以下に示します。例では、RegistrationタスクでPFHを使っています。


In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
from tutlibs.registration import brute_force_matching
from tutlibs.normal_estimation import normal_estimation
from tutlibs.transformation import translation, rotation
from tutlibs.sampling import voxel_grid_sampling
from tutlibs.feature import pfh
from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer

In [13]:
# data acquisition
source_xyz, source_colors, _ = io.read('../data/pcl_data/tutorials/correspondence_grouping/milk_cartoon_all_small_clorox.pcd')
target_xyz, target_colors, _ = io.read('../data/pcl_data/tutorials/correspondence_grouping/milk.pcd')

# target random transformation
target_xyz = rotation(translation(target_xyz, np.array([1.5, 0, 0])), 'y', angle=45)

# keypoints estimation
source_dsxyz = voxel_grid_sampling(source_xyz, 0.05)
target_dsxyz = voxel_grid_sampling(target_xyz, 0.05)

# feature descriptors estimation
source_normals = normal_estimation(source_dsxyz)
target_normals = normal_estimation(target_dsxyz)
sources_pfh = pfh(source_dsxyz, source_normals)
target_pfh = pfh(target_dsxyz, target_normals)

# correspondences estiomation (matching)
match_idxs = brute_force_matching(source_dsxyz, target_dsxyz)

# correspondence rejection method
ransac(source_dsxyz, target_dsxyz, sources_pfh, target_pfh)



Output()

## Methods with Deep learning
Deep Learningを用いたpoint feature representationの取得では、Deep learningによって特徴抽出に最適なパラメータを自動取得し、そのパラメータに基づいて抽出を行います。この手法は他分野のDeep learning同様、目的のタスクに合うデータを用意する必要があるなどいくつかの制限がありますが、制限をクリアできれば他の手法よりも良い結果を得る可能性があります。そのため、近年(2021年)ではこのDeep learningを用いた手法が盛んに研究されています。

例としては、

その一例として、PointNet++を使ったRegistrationタスクを行ってみましょう。

In [ ]:
# PointNet++

こちらでは、丸々のようなものが使われています.

rNN
sampling
upsampling?
conv1D

関連資料
- 各モジュール
- 点群特性




## References
- [Rusu, R.B. Semantic 3D Object Maps for Everyday Manipulation in Human Living Environments. Künstl Intell 24, 345–348 (2010). https://doi.org/10.1007/s13218-010-0059-6](https://link.springer.com/article/10.1007/s13218-010-0059-6#citeas)
- [R. B. Rusu, N. Blodow and M. Beetz, "Fast Point Feature Histograms (FPFH) for 3D registration," 2009 IEEE International Conference on Robotics and Automation, 2009, pp. 3212-3217, doi: 10.1109/ROBOT.2009.5152473.](https://www.cvl.iis.u-tokyo.ac.jp/class2016/2016w/papers/6.3DdataProcessing/Rusu_FPFH_ICRA2009.pdf)
- [Rusu, Radu Bogdan, Zoltan Csaba Marton, Nico Blodow, and Michael Beetz. 2008. “Learning Informative Point Classes for the Acquisition of Object Model Maps.” In 2008 10th International Conference on Control, Automation, Robotics and Vision. IEEE. https://doi.org/10.1109/icarcv.2008.4795593.](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.584.2647&rep=rep1&type=pdf)
- [Point Cloud Library. Point Feature Histograms (PFH) descriptors. URL: https://pcl.readthedocs.io/projects/tutorials/en/latest/pfh_estimation.html?highlight=PFH (Access: 2021/09/24)](https://pcl.readthedocs.io/projects/tutorials/en/latest/pfh_estimation.html?highlight=PFH)